# 1. Setup

In [37]:
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras import models
import tensorflow as tf
import os
from keras.models import load_model

In [38]:
class WeightedAverageLayer(tf.keras.layers.Layer):
    def __init__(self, weight1, weight2, **kwargs):
        super(WeightedAverageLayer, self).__init__(**kwargs)
        self.w1 = weight1
        self.w2 = weight2

    def get_config(self):
        config = super().get_config()
        config.update({
            "weight1": self.w1,
            "weight2": self.w2,
        })
        return config

    def call(self, inputs):
        return self.w1 * inputs[0] + self.w2 * inputs[1]

In [39]:
input_height = 69
input_width = 69
input_depth = 3
batch_size = 64

In [40]:
paths = {
    'CNN_DIR' : os.path.join('models', 'CNN_dropout_batch_bigger_class_weights.h5'),
    'VGG_DIR' : os.path.join('models', 'VGG16_finetuned.h5'),
    'TEST_PATH' : os.path.join('workspace','images', 'test')
 }

In [41]:
cnn = load_model(paths['CNN_DIR'],compile=False)
vgg = load_model(paths['VGG_DIR'],compile=False)

In [42]:
models = [cnn,vgg]

In [43]:
#rename model to prevent conflicts later
for i, layer in enumerate(cnn.layers):
    layer._name = layer._name + "_cnn"

cnn._name = cnn._name + "_cnn"

In [44]:
cnn.summary()

Model: "sequential_cnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_cnn (Conv2D)         (None, 69, 69, 16)        448       
                                                                 
 batch_normalization_cnn (Ba  (None, 69, 69, 16)       64        
 tchNormalization)                                               
                                                                 
 max_pooling2d_cnn (MaxPooli  (None, 34, 34, 16)       0         
 ng2D)                                                           
                                                                 
 conv2d_1_cnn (Conv2D)       (None, 34, 34, 32)        4640      
                                                                 
 batch_normalization_1_cnn (  (None, 34, 34, 32)       128       
 BatchNormalization)                                             
                                                    

In [45]:
def ensemble(weight1 = 0.5, weight2 = 0.5):
    model_input = tf.keras.Input(shape=(input_width, input_height, input_depth))
    model_outputs = [model(model_input) for model in models]
    # if weights are equal, it's the same as calling keras.layers.Average()
    ensemble_output = WeightedAverageLayer(weight1,weight2)(model_outputs)
    ensemble_model = tf.keras.Model(inputs=model_input, outputs=ensemble_output)

    return ensemble_model

# 2. Model testing

In [46]:
number_test = 0
for i in range(9):
      path = os.path.join(paths['TEST_PATH'],str(i))
      n_images = len([f for f in os.listdir(path)if os.path.isfile(os.path.join(path, f))])
      number_test += n_images

In [47]:
test_dir=paths['TEST_PATH']
print(test_dir)
test_datagen = ImageDataGenerator(rescale=1./255) # it should not be augmented

test_generator = test_datagen.flow_from_directory(test_dir, target_size=(input_width, input_height), batch_size=batch_size, class_mode='categorical', classes=None, shuffle=False)

workspace/images/test
Found 2931 images belonging to 9 classes.


# 3. Find best weights

In [ ]:
import numpy as np
from sklearn.metrics import  classification_report
import csv

with open('results.csv','a') as file:
    writer = csv.writer(file)
    header = ['w1', 'w2', 'avg_precision', 'avg_recall','avg_f1score','accuracy']
    writer.writerow(header)
    w1 = 0.0
    while w1 <= 1:
        w2 = 1.0 - w1
        Y_pred = ensemble(w1,w2).predict(test_generator, number_test // batch_size+1)
        y_pred = np.argmax(Y_pred, axis=1)
        target_names = ['0','1','2','3','4','5','6','7','8']
        report = classification_report(test_generator.classes, y_pred, target_names=target_names,output_dict=True)
        writer.writerow([str(w1),str(w2),report['macro avg']['precision'],report['macro avg']['recall'],report['macro avg']['f1-score'],report['accuracy']])
        w1 += 0.1

# 4. Plot metrics with best weights

In [49]:
ensemble_net = ensemble(0.6,0.4)

In [ ]:
Y_pred = ensemble_net.predict(test_generator, number_test // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, roc_auc_score, auc
fig, c_ax = plt.subplots(1,1, figsize = (12, 8))

def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)

    for (idx, c_label) in enumerate(target_names):
        fpr, tpr, thresholds = roc_curve(y_test[:,idx].astype(int), y_pred[:,idx])
        c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
    c_ax.plot(fpr, fpr, 'b-', label = 'Random Guessing')
    c_ax.legend()
    return roc_auc_score(y_test, y_pred, average=average)


test_generator.reset()
y_pred = ensemble_net.predict(test_generator, verbose = True)
y_pred = np.argmax(y_pred, axis=1)
print("Multiclass roc auc score:", multiclass_roc_auc_score(test_generator.classes, y_pred))

# 5. Model exportation

In [ ]:
ensemble_net.save("models/ensamble_model.h5")

# 6. Plot model as graph of layers

In [ ]:
from keras.utils import plot_model

In [ ]:
plot_model(ensemble_net, show_shapes=True, to_file='model_ensemble.png')